In [1]:
!apt-get update && apt-get install -y poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]                           
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [69.9 kB]       
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]              
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,767 kB]  
Get:11 http://archive.ubuntu.com/ubuntu jam

In [2]:
from pdf2image import convert_from_path
import pytesseract
import json

pdf_path = '/kaggle/input/vertx-a-q2/Mr-Ratan-Tata-Startup-Pitch-Deck-Template.pdf'

# Convert PDF pages to images at a high resolution for better OCR accuracy.
pages = convert_from_path(pdf_path, dpi=300)

text = ''
for page in pages:
    text += pytesseract.image_to_string(page)

# Save as a plain text file.
with open('extracted_text_3.txt', 'w', encoding='utf-8') as txt_file:
    txt_file.write(text)

# Save as a JSON file.
# data = {"extracted_text": text}
# with open('extracted_text.json', 'w', encoding='utf-8') as json_file:
#     json.dump(data, json_file, ensure_ascii=False, indent=4)

print("Extraction completed and saved to 'extracted_text.txt' and 'extracted_text.json'.")

Extraction completed and saved to 'extracted_text.txt' and 'extracted_text.json'.


In [3]:
import re

def preprocess_text(text):
    # Remove form feed characters and other unwanted control characters.
    text = text.replace('\x0c', ' ')
    
    # Normalize line breaks: replace multiple line breaks with a single one.
    text = re.sub(r'\n+', '\n', text)
    
    # Remove leading and trailing whitespace from each line.
    lines = [line.strip() for line in text.splitlines()]
    
    # Optionally, remove lines that are likely artifacts (e.g., lines with only non-alphanumeric characters)
    lines = [line for line in lines if re.search(r'\w', line)]
    
    # Join lines into paragraphs.
    cleaned_text = "\n".join(lines)
    
    # Additional cleaning: remove multiple spaces.
    cleaned_text = re.sub(r' {2,}', ' ', cleaned_text)
    
    return cleaned_text.strip()

# Read the OCR output from a file.
with open('/kaggle/working/extracted_text_3.txt', 'r', encoding='utf-8') as file:
    ocr_text = file.read()

# Apply preprocessing to the text.
cleaned_text = preprocess_text(ocr_text)

# Print the cleaned text.
# print(cleaned_text)

# Save the cleaned output to a new file.
with open('cleaned_output_3.txt', 'w', encoding='utf-8') as file:
    file.write(cleaned_text)

print("Cleaned text has been saved to 'cleaned_output.txt'.")

Cleaned text has been saved to 'cleaned_output.txt'.


In [4]:
from google import genai

# ------------------------------
# CHANGE THIS: Set your actual API key here.
API_KEY = "YOUR_API_KEY"
client = genai.Client(api_key=API_KEY)
# ------------------------------

# Predefined section weights (adjust if needed)
section_weights = {
    "Problem": 15,
    "Solution": 15,
    "Market": 20,
    "Business Model": 20,
    "Financials": 20,
    "Team": 10
}

def extract_section(section_name, full_text):
    """
    Extract text corresponding to a specific section using Gemini.
    Adjust the prompt if necessary.
    """
    prompt = f"""
    You are an expert in analyzing startup pitch decks. 
    Given the following pitch deck content, please extract and return the text that best represents the "{section_name}" section.
    Focus on capturing details relevant to {section_name.lower()} aspects (e.g., customer pain points for Problem, unique solution for Solution, market size/trends for Market, revenue streams for Business Model, projections for Financials, and team experience for Team).
    
    Pitch Deck Content:
    {full_text}
    
    If there is no clear information for this section, return an empty response.
    """
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt]
    )
    return response.text.strip()

def evaluate_section(section_name, section_text):
    """
    Evaluate the quality of the given section using Gemini.
    The API should return a score between 0 and 100.
    Adjust the prompt if needed.
    """
    prompt = f"""
    You are an expert evaluator of startup pitch decks. 
    Please evaluate the quality of the "{section_name}" section of a pitch deck.
    Consider the following metrics:
      1. Clarity
      2. Relevance
      3. Depth
      4. Persuasiveness
      5. Structure

    Provide a single numeric score between 0 and 100, where 100 indicates an excellent section and 0 indicates a very poor section.

    Section Content:
    {section_text}

    Only output the numeric score.
    """
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt]
    )
    score_text = response.text.strip()
    try:
        # Attempt to extract a numeric score from the response.
        score = int(''.join(filter(str.isdigit, score_text)))
    except Exception:
        score = 0  # Default to 0 if conversion fails.
    return score

# ------------------------------
# CHANGE THIS: Read your cleaned pitch deck text from a file.
with open('/kaggle/working/cleaned_output_3.txt', 'r', encoding='utf-8') as file:
    cleaned_text = file.read()
# ------------------------------

# Define the sections we want to extract and evaluate.
sections_to_extract = ["Problem", "Solution", "Market", "Business Model", "Financials", "Team"]

# Step 1: Extract the text for each section from the full pitch deck.
extracted_sections = {}
for section in sections_to_extract:
    extracted_text = extract_section(section, cleaned_text)
    extracted_sections[section] = extracted_text

# Save the extracted text to a file named "LLM_extracted_text.txt".
with open('LLM_extracted_text_3.txt', 'w', encoding='utf-8') as file:
    for section in sections_to_extract:
        file.write(f"Section: {section}\n")
        file.write(extracted_sections[section] + "\n")
        file.write("-" * 40 + "\n")

print("Extracted text has been saved to 'LLM_extracted_text.txt'.")

# Step 2: Evaluate each extracted section to get a score.
section_scores = {}
for section in sections_to_extract:
    score = evaluate_section(section, extracted_sections[section])
    section_scores[section] = score
    print(f"{section} Score: {score}")

# Step 3: Calculate the weighted final pitch score (0-100).
total_weight = sum(section_weights.values())
weighted_sum = sum(
    section_scores[sec] * (section_weights[sec] / total_weight)
    for sec in section_scores
)
# print("\nFinal Pitch Score:", weighted_sum)


Extracted text has been saved to 'LLM_extracted_text.txt'.
Problem Score: 40
Solution Score: 50
Market Score: 45
Business Model Score: 25
Financials Score: 65
Team Score: 65


In [5]:
def generate_feedback(section_scores, final_pitch_score):
    """
    Use the Gemini API to generate a detailed strength and weakness analysis
    along with personalized content improvement suggestions.
    """
    prompt = f"""
    You are an expert evaluator of startup pitch decks. Based on the following section scores and evaluations:

    Section Scores:
    {section_scores}

    The final weighted pitch score is {final_pitch_score} out of 100.

    Please provide a detailed strength and weakness analysis of the pitch deck. For each section (Problem, Solution, Market, Business Model, Financials, and Team), offer personalized feedback on areas that need improvement. Also, suggest specific content improvements or additional data that could be included.

    Finally, provide an overall summary of the pitch deck's strengths and weaknesses. Format your response in clear bullet points or numbered lists where appropriate.
    """
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt]
    )
    return response.text.strip()

In [6]:
# Step 4: Generate AI-based feedback with a Strength & Weakness Analysis.
feedback = generate_feedback(section_scores, weighted_sum)
print("\nFeedback:")
# print(feedback)

# Save the feedback to a text file.
with open("feedback.txt_3", "w", encoding="utf-8") as feedback_file:
    feedback_file.write(feedback)

print("Feedback has been saved to 'feedback.txt'.")


Feedback:
Feedback has been saved to 'feedback.txt'.
